In [1]:
import pandas as pd
from pulp import *
import numpy as np
import math
import random
from time import*
from datetime import *
import time
import csv
import matplotlib.pyplot as plt
from gurobipy import *

In [2]:
demands = range(0, 426) ## demand nodes. Belo Horizonte’s districts represented by the geometric centre of each one
location_sites = range(0, 1112) ## The set of potential ambulance location sites - base station.
periods = 1 ## The set of periods T considered
amb_type = range(0, 2) ## ALS and BLS
max_num_bases = 25 ## max numer of bases to be installed
amb_disp = {0: 7, 1: 21} ## The number of vehicles available Pu is 7 ALS and 21 BLS.
P = pd.Series(data=amb_disp) ##amb_disp
resp_time = {0: 10, 1: 8} ## EMS’s response time standard is 10 minutes for ALS and 8 minutes for BLS
S = pd.Series(data=resp_time) ## resp_time
other_info_a = pd.read_excel('FLEET_ICT_DATASET.xlsx', 'STATION_CAPACITY')
C = other_info_a['STATION_CAPACITY'] ## station capacity
other_info_b = pd.read_excel('FLEET_ICT_DATASET.xlsx', 'DISTANCE_MATRIX')
d = {}
for index, linha in other_info_b.iterrows():
    base = linha['STATION']
    demand_point = linha['DEMAND_POINT']
    distance = linha['DISTANCE']
    
    if base not in d:
        d[base] = {}

    d[base][demand_point] = distance
    
other_info_c = pd.read_excel('FLEET_ICT_DATASET.xlsx', 'DEMANDS')
    
# Transformar os dados em uma estrutura de índices
q = other_info_c.set_index('DEMAND_POINT')[['USA_DEMAND', 'USB_DEMAND']]

# Converter para dicionário
q_dict = q.to_dict()

# Mapear índices desejados para os valores correspondentes
mapped_dict = {}
for i, col in enumerate(q.columns):
    mapped_dict[i] = q_dict[col] 

In [3]:
# solver GLPK as defaut
solver = GLPK()

# Creating the model
model2 = LpProblem("Maximize_Demand_Coverage", LpMaximize)

# Defining variables
x = LpVariable.dicts("base_to_ambulance_type", (location_sites, amb_type), 0, 1, LpBinary)
# RELAXING y VARIABLE TRYING TO MAKE THE RESOLUTION FASTER WITHOUT CHANGING THE RESULTS
y = LpVariable.dicts("demand_coverage", (demands, amb_type), 0, 1, LpContinuous)
z = LpVariable.dicts("base_allocation", location_sites, 0, 1, LpBinary)

# Objective function >> with LpAffineExpression
model2 += pulp.LpAffineExpression([(y[i][0], q.iloc[i, 0]) for i in demands]) + pulp.LpAffineExpression([(y[i][1], q.iloc[i, 1]) for i in demands])

# Constraint_01: each demand point i is considered covered for any ambulance type u if and only there is an ambulance
# type u allocated at station j which is whithin the response time limit for ambulance type u
for i in demands:
    for u in amb_type:
        model2 += lpSum([x[j][u] for j in location_sites if d[j][i] <= S[u]]) >= y[i][u]

# Constraint_02: the number of ambulances assigned must respect the number of resorces available for each ambulance type u
for u in amb_type:
    model2 += lpSum([x[j][u] for j in location_sites]) <= P[u]

# Constraint_03: each station zj have its own capacity to allocate Cj veihcles 
for j in location_sites:
    model2 += lpSum([x[j][u] for u in amb_type]) <= C[j] * z[j]

# Constraint_04: garantee that ambulances will be allocated only in active stations
for j in location_sites:
    for u in amb_type:
        model2 += [x[j][u]] <= z[j]

# Constraint_05: limits the number of stations (bases) to be assigned
model2 += lpSum([z[j] for j in location_sites]) <= max_num_bases

# Constraint_06: EMS operational center as a choosen station
model2 += z[1053] == 1

model2.solver = solver

model2.solve()

# Displaying information
Status2 = LpStatus[model2.status]
execution_time2 = (model2.solutionTime / 60)
objective_function_value2 = value(model2.objective)

print('Total demand coverage:', objective_function_value2, '// Status:', Status2, '// Execution time:', execution_time2, '// Objective Function:', objective_function_value2)
print('Solver:', model2.solver)

Total demand coverage: 23281.0 // Status: Optimal // Execution time: 9.018520220120747 // Objective Function: 23281.0
Solver: <pulp.apis.glpk_api.GLPK_CMD object at 0x0000022BE524F7F0>


In [4]:
# Saving z results in an Excel sheet
df_z = pd.DataFrame({"Variable": [], "Value": []})
for j in location_sites:
    if z[j].varValue > 0:
        df_z = df_z.append(pd.DataFrame({"Variable": [f"z_{j}"], "Value": [z[j].varValue]}))

# Saving x results in another Excel sheet
df_x = pd.DataFrame({"Variable": [], "Value": []})
for j in location_sites:
    for u in amb_type:
        if x[j][u].varValue > 0:
            df_x = df_x.append(pd.DataFrame({"Variable": [f"x_{j}_{u}"], "Value": [x[j][u].varValue]}))

# Saving y results in another Excel sheet
df_y = pd.DataFrame({"Variable": [], "Value": [], "Demand_Coverage": []})
for u in amb_type:
    for i in demands:
        if y[i][u].varValue > 0:
            demand_col = q.iloc[i, u]
            df_y = df_y.append(pd.DataFrame({"Variable": [f"y_{i}_{u}"], "Value": [y[i][u].varValue], "Demand_Coverage": [demand_col]}))

# grouping all tree variable results in one XLSX file
with pd.ExcelWriter("variaveis2.xlsx") as writer:
    df_z.to_excel(writer, sheet_name='Variables_Z', index=False)
    df_x.to_excel(writer, sheet_name='Variables_X', index=False)
    df_y.to_excel(writer, sheet_name='Variables_Y', index=False)

In [5]:
# solver GLPK as defaut
solver = GLPK()

# Creating the model
model2 = LpProblem("Maximize_Demand_Coverage", LpMaximize)

# Defining variables >>> # RELAXING x,y and z VARIABLES TRYING TO MAKE THE RESOLUTION FASTER WITHOUT CHANGING THE RESULTS
x = LpVariable.dicts("base_to_ambulance_type", (location_sites, amb_type), 0, 1, LpContinuous)
y = LpVariable.dicts("demand_coverage", (demands, amb_type), 0, 1, LpContinuous)
z = LpVariable.dicts("base_allocation", location_sites, 0, 1, LpBinary)

# Objective function >> with LpAffineExpression
model2 += pulp.LpAffineExpression([(y[i][0], q.iloc[i, 0]) for i in demands]) + pulp.LpAffineExpression([(y[i][1], q.iloc[i, 1]) for i in demands])

# Constraint_01: each demand point i is considered covered for any ambulance type u if and only there is an ambulance
# type u allocated at station j which is whithin the response time limit for ambulance type u
for i in demands:
    for u in amb_type:
        model2 += lpSum([x[j][u] for j in location_sites if d[j][i] <= S[u]]) >= y[i][u]

# Constraint_02: the number of ambulances assigned must respect the number of resorces available for each ambulance type u
for u in amb_type:
    model2 += lpSum([x[j][u] for j in location_sites]) <= P[u]

# Constraint_03: each station zj have its own capacity to allocate Cj veihcles 
for j in location_sites:
    model2 += lpSum([x[j][u] for u in amb_type]) <= C[j] * z[j]

# Constraint_04: garantee that ambulances will be allocated only in active stations
for j in location_sites:
    for u in amb_type:
        model2 += [x[j][u]] <= z[j]

# Constraint_05: limits the number of stations (bases) to be assigned
model2 += lpSum([z[j] for j in location_sites]) <= max_num_bases

# Constraint_06: EMS operational center as a choosen station
model2 += z[1053] == 1

model2.solver = solver

model2.solve()

# Displaying information
Status2 = LpStatus[model2.status]
execution_time2 = (model2.solutionTime / 60)
objective_function_value2 = value(model2.objective)

print('Total demand coverage:', objective_function_value2, '// Status:', Status2, '// Execution time:', execution_time2, '// Objective Function:', objective_function_value2)
print('Solver:', model2.solver)

Total demand coverage: 23281.0 // Status: Optimal // Execution time: 25.23674731651942 // Objective Function: 23281.0
Solver: <pulp.apis.glpk_api.GLPK_CMD object at 0x0000022BE3D28760>


In [6]:
# Saving z results in an Excel sheet
df_z = pd.DataFrame({"Variable": [], "Value": []})
for j in location_sites:
    if z[j].varValue > 0:
        df_z = df_z.append(pd.DataFrame({"Variable": [f"z_{j}"], "Value": [z[j].varValue]}))

# Saving x results in another Excel sheet
df_x = pd.DataFrame({"Variable": [], "Value": []})
for j in location_sites:
    for u in amb_type:
        if x[j][u].varValue > 0:
            df_x = df_x.append(pd.DataFrame({"Variable": [f"x_{j}_{u}"], "Value": [x[j][u].varValue]}))

# Saving y results in another Excel sheet
df_y = pd.DataFrame({"Variable": [], "Value": [], "Demand_Coverage": []})
for u in amb_type:
    for i in demands:
        if y[i][u].varValue > 0:
            demand_col = q.iloc[i, u]
            df_y = df_y.append(pd.DataFrame({"Variable": [f"y_{i}_{u}"], "Value": [y[i][u].varValue], "Demand_Coverage": [demand_col]}))

# grouping all tree variable results in one XLSX file
with pd.ExcelWriter("variaveis3.xlsx") as writer:
    df_z.to_excel(writer, sheet_name='Variables_Z', index=False)
    df_x.to_excel(writer, sheet_name='Variables_X', index=False)
    df_y.to_excel(writer, sheet_name='Variables_Y', index=False)

In [7]:
# solver GLPK as defaut
solver = GLPK()

# Creating the model
model2 = LpProblem("Maximize_Demand_Coverage", LpMaximize)

# Defining variables >>> # RELAXING x and z VARIABLES TRYING TO MAKE THE RESOLUTION FASTER WITHOUT CHANGING THE RESULTS
x = LpVariable.dicts("base_to_ambulance_type", (location_sites, amb_type), 0, 1, LpBinary)
y = LpVariable.dicts("demand_coverage", (demands, amb_type), 0, 1, LpContinuous)
z = LpVariable.dicts("base_allocation", location_sites, 0, 1, LpContinuous)

# Objective function >> with LpAffineExpression
model2 += pulp.LpAffineExpression([(y[i][0], q.iloc[i, 0]) for i in demands]) + pulp.LpAffineExpression([(y[i][1], q.iloc[i, 1]) for i in demands])

# Constraint_01: each demand point i is considered covered for any ambulance type u if and only there is an ambulance
# type u allocated at station j which is whithin the response time limit for ambulance type u
for i in demands:
    for u in amb_type:
        model2 += lpSum([x[j][u] for j in location_sites if d[j][i] <= S[u]]) >= y[i][u]

# Constraint_02: the number of ambulances assigned must respect the number of resorces available for each ambulance type u
for u in amb_type:
    model2 += lpSum([x[j][u] for j in location_sites]) <= P[u]

# Constraint_03: each station zj have its own capacity to allocate Cj veihcles 
for j in location_sites:
    model2 += lpSum([x[j][u] for u in amb_type]) <= C[j] * z[j]

# Constraint_04: garantee that ambulances will be allocated only in active stations
for j in location_sites:
    for u in amb_type:
        model2 += [x[j][u]] <= z[j]

# Constraint_05: limits the number of stations (bases) to be assigned
model2 += lpSum([z[j] for j in location_sites]) <= max_num_bases

# Constraint_06: EMS operational center as a choosen station
model2 += z[1053] == 1

model2.solver = solver

model2.solve()

# Displaying information
Status2 = LpStatus[model2.status]
execution_time2 = (model2.solutionTime / 60)
objective_function_value2 = value(model2.objective)

print('Total demand coverage:', objective_function_value2, '// Status:', Status2, '// Execution time:', execution_time2, '// Objective Function:', objective_function_value2)
print('Solver:', model2.solver)

Total demand coverage: 23281.0 // Status: Optimal // Execution time: 1.7436758915583292 // Objective Function: 23281.0
Solver: <pulp.apis.glpk_api.GLPK_CMD object at 0x0000022BDD2AD9D0>


In [8]:
# Saving z results in an Excel sheet
df_z = pd.DataFrame({"Variable": [], "Value": []})
for j in location_sites:
    if z[j].varValue > 0:
        df_z = df_z.append(pd.DataFrame({"Variable": [f"z_{j}"], "Value": [z[j].varValue]}))

# Saving x results in another Excel sheet
df_x = pd.DataFrame({"Variable": [], "Value": []})
for j in location_sites:
    for u in amb_type:
        if x[j][u].varValue > 0:
            df_x = df_x.append(pd.DataFrame({"Variable": [f"x_{j}_{u}"], "Value": [x[j][u].varValue]}))

# Saving y results in another Excel sheet
df_y = pd.DataFrame({"Variable": [], "Value": [], "Demand_Coverage": []})
for u in amb_type:
    for i in demands:
        if y[i][u].varValue > 0:
            demand_col = q.iloc[i, u]
            df_y = df_y.append(pd.DataFrame({"Variable": [f"y_{i}_{u}"], "Value": [y[i][u].varValue], "Demand_Coverage": [demand_col]}))

# grouping all tree variable results in one XLSX file
with pd.ExcelWriter("variaveis4.xlsx") as writer:
    df_z.to_excel(writer, sheet_name='Variables_Z', index=False)
    df_x.to_excel(writer, sheet_name='Variables_X', index=False)
    df_y.to_excel(writer, sheet_name='Variables_Y', index=False)